In [1]:
import pandas as pd
file_path = '/work/teghipco/LLMs/Biobert/pubs_all.parquet'
df = pd.read_parquet(file_path)

In [2]:
df.shape

(2979275, 30)

In [3]:
print(df.head())

   publication_id  year        date date_online  \
0  pub.1142700518  2021  2021-11-19  2021-11-19   
1  pub.1157734354  2023  2023-05-04  2023-05-04   
2  pub.1176144398  2024  2024-09-30  2024-09-30   
3  pub.1133314038  2020  2020-11-01        None   
4  pub.1168937635  2024  2024-02-06        None   

                                     title_preferred clinical_trial_ids  \
0  P11‐17: Outcomes of COVID 19 critical care pat...                      
1  The Bronte Creek Project: Outdoor Environmenta...                      
2  (Mis)recognising the symbolic violence of acad...                      
3                                  Lunar dust buster                      
4               Lessons from COVID-19: UK experience                      

  times_cited recent_citations publication_type  is_open_access  ...  \
0           0                0          article            True  ...   
1           0                0          chapter            True  ...   
2           0             

In [4]:
columns_to_drop = ['title_preferred', 'proceedings_title_preferred', 'conference_name', 'title_original']
df = df.drop(columns=columns_to_drop)

In [5]:
print(df.head())

   publication_id  year        date date_online clinical_trial_ids  \
0  pub.1142700518  2021  2021-11-19  2021-11-19                      
1  pub.1157734354  2023  2023-05-04  2023-05-04                      
2  pub.1176144398  2024  2024-09-30  2024-09-30                      
3  pub.1133314038  2020  2020-11-01        None                      
4  pub.1168937635  2024  2024-02-06        None                      

  times_cited recent_citations publication_type  is_open_access  \
0           0                0          article            True   
1           0                0          chapter            True   
2           0                0          article            True   
3           0                0          article            True   
4           0                0          chapter            True   

                                             authors  ...  \
0  [{'author_grid_ids': [], 'first_name': None, '...  ...   
1  [{'author_grid_ids': [], 'first_name': 'John',...  

In [6]:
columns_to_drop = ['abstract_preferred']
df = df.drop(columns=columns_to_drop)

In [8]:
columns_to_drop = ['book_title_preferred']
df = df.drop(columns=columns_to_drop)

In [9]:
print(df.head())

   publication_id  year        date date_online clinical_trial_ids  \
0  pub.1142700518  2021  2021-11-19  2021-11-19                      
1  pub.1157734354  2023  2023-05-04  2023-05-04                      
2  pub.1176144398  2024  2024-09-30  2024-09-30                      
3  pub.1133314038  2020  2020-11-01        None                      
4  pub.1168937635  2024  2024-02-06        None                      

  times_cited recent_citations publication_type  is_open_access  \
0           0                0          article            True   
1           0                0          chapter            True   
2           0                0          article            True   
3           0                0          article            True   
4           0                0          chapter            True   

                                             authors  ...  \
0  [{'author_grid_ids': [], 'first_name': None, '...  ...   
1  [{'author_grid_ids': [], 'first_name': 'John',...  

In [10]:
df['num_authors'] = df['authors'].apply(lambda x: len(x) if isinstance(x, list) else 0)
df['is_open_access'] = df['is_open_access'].astype(int)

MemoryError: Unable to allocate 193. GiB for an array with shape (2979275, 69410) and data type bool

In [13]:
# One-hot encode publication_type
df = pd.get_dummies(df, columns=['publication_type'], drop_first=True)

KeyError: "None of [Index(['publication_type'], dtype='object')] are in the [columns]"

In [16]:
print(df.columns)

Index(['publication_id', 'year', 'clinical_trial_ids', 'times_cited',
       'recent_citations', 'is_open_access', 'authors', 'funding_details',
       'unique_org_ids', 'research_org_city_ids', 'research_org_country_codes',
       'research_org_state_codes', 'num_unique_orgs', 'altmetric_score',
       'field_citation_ratio', 'field', 'citation_count',
       'relative_citation_ratio', 'dataset_count', 'grant_count',
       'patent_count', 'num_authors', 'publication_type_book',
       'publication_type_chapter', 'publication_type_monograph',
       'publication_type_preprint', 'publication_type_proceeding',
       'publication_type_seminar', 'date_diff_days'],
      dtype='object')


In [17]:
from datetime import datetime
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['date_online'] = pd.to_datetime(df['date_online'], errors='coerce')

KeyError: 'date'

In [18]:
df = df.dropna(subset=['altmetric_score'])

In [19]:
# Fill missing values for numeric columns with 0 or the median, depending on context
df['field_citation_ratio'] = df['field_citation_ratio'].fillna(0)
df['citation_count'] = df['citation_count'].fillna(0)
df['relative_citation_ratio'] = df['relative_citation_ratio'].fillna(0)

In [20]:
# Fill categorical columns with a placeholder if necessary
df['research_org_state_codes'] = df['research_org_state_codes'].fillna('Unknown')
# One-hot encode research_org_state_codes
df = pd.get_dummies(df, columns=['research_org_state_codes'], drop_first=True)

MemoryError: Unable to allocate 193. GiB for an array with shape (2979275, 69410) and data type bool

In [21]:
print(df.columns)

Index(['publication_id', 'year', 'clinical_trial_ids', 'times_cited',
       'recent_citations', 'is_open_access', 'authors', 'funding_details',
       'unique_org_ids', 'research_org_city_ids', 'research_org_country_codes',
       'research_org_state_codes', 'num_unique_orgs', 'altmetric_score',
       'field_citation_ratio', 'field', 'citation_count',
       'relative_citation_ratio', 'dataset_count', 'grant_count',
       'patent_count', 'num_authors', 'publication_type_book',
       'publication_type_chapter', 'publication_type_monograph',
       'publication_type_preprint', 'publication_type_proceeding',
       'publication_type_seminar', 'date_diff_days'],
      dtype='object')


In [25]:
# Count the number of clinical trial IDs by splitting the string
df['num_clinical_trial_ids'] = df['clinical_trial_ids'].apply(lambda x: len(x.split(',')) if pd.notnull(x) else 0)

In [26]:
print(df['num_clinical_trial_ids'].unique())    # Displays unique values, including any non-zero counts

[  1   2   4   6   8  16  11  22  68   3  84   7  13  27  21  20  61   5
  34  15   9  55  29  63  72  25  87  73  17  23  10  18  56  19  53  58
  35  33  12  36  32  24  49  14  71  44  52 104  43  48 137  40 127  38
  79  74  67  26  80  28  83  45  31  30 118  57  39  54  82  66 214  93
  91 112  86  37  46 124  88 122  42  96  92 154  70 217  41  95 119  51
  59  60  81  47  85  76  75 115 108 142  62 116 173 125  78 106  89  77
 121 232 117 178 248 231 401  50 212  65  64 188 149 169 183 139 235  69
 126 384 145 101 207  97 210 103 204 194 350 165  90 174 159 120 135 211
 284 143 146 113 134 412 195 260 153  99 409 203 177 185  98 128 357 107
 111 102 167 270 161 155 151 486 247 171 105 763 114 236 160 168 147 110
 304 109 184 176 377 249 315 162 131 123 130 234 498 221 313 179 295 133
 136  94 261 216 201 156 140 144 287 291 190 189 506 293 253 100 157 209
 297 186 175 198 181 250 299 391 152 282 289 292 200 180 138 129]


In [33]:
df['num_unique_city_ids'] = df['research_org_city_ids'].apply(lambda x: len(set(x.split(','))) if pd.notnull(x) else 0)

In [35]:
df['num_unique_country_codes'] = df['research_org_country_codes'].apply(lambda x: len(set(x.split(','))) if pd.notnull(x) else 0)

In [37]:
df['num_unique_state_codes'] = df['research_org_state_codes'].apply(lambda x: len(set(x.split(','))) if pd.notnull(x) else 0)


In [38]:
df = df.drop(columns=['clinical_trial_ids','field','research_org_state_codes','funding_details','unique_org_ids','research_org_city_ids','research_org_country_codes','research_org_state_codes'])

In [36]:
print(df.num_unique_country_codes)

0          1
1          1
2          1
3          1
4          1
          ..
2979270    1
2979271    2
2979272    1
2979273    1
2979274    1
Name: num_unique_country_codes, Length: 2979275, dtype: int64


In [39]:
print(df.columns)

Index(['publication_id', 'year', 'times_cited', 'recent_citations',
       'is_open_access', 'authors', 'num_unique_orgs', 'altmetric_score',
       'field_citation_ratio', 'citation_count', 'relative_citation_ratio',
       'dataset_count', 'grant_count', 'patent_count', 'num_authors',
       'publication_type_book', 'publication_type_chapter',
       'publication_type_monograph', 'publication_type_preprint',
       'publication_type_proceeding', 'publication_type_seminar',
       'date_diff_days', 'num_clinical_trial_ids', 'num_unique_city_ids',
       'num_unique_country_codes', 'num_unique_state_codes'],
      dtype='object')


In [40]:
# Define features and target
X = df.drop(columns=['altmetric_score'])  # Drop target column from features
y = df['altmetric_score']  # Define target

In [41]:
X.shape

(2979275, 25)

In [42]:
# Separate out pub_id and authors into their own variables for future reference
pub_id = df['publication_id']
authors_info = df['authors']

# Drop 'publication_id' and 'authors' from the feature set
X = df.drop(columns=['publication_id', 'authors'])

In [45]:
X.to_parquet("X_features.parquet")
y.to_frame(name='altmetric_score').to_parquet("y_target.parquet")  # Convert y to a DataFrame first
pub_id.to_frame(name='pub_id').to_parquet("pub_id.parquet")
authors_info.to_frame(name='authors_info').to_parquet("authors_info.parquet")

In [47]:
# List of variables you want to keep
keep_vars = ['X', 'y', 'pub_id', 'authors_info']

# Delete all other variables except those in keep_vars
for var in list(globals().keys()):
    if var not in keep_vars and not var.startswith("__"):
        del globals()[var]


In [75]:
from xgboost import XGBRegressor
from skopt import BayesSearchCV
from skopt.space import Real, Integer
from sklearn.model_selection import GroupKFold, TimeSeriesSplit
from sklearn.metrics import mean_absolute_error
import numpy as np
import pandas as pd

In [94]:
# Explicitly use .loc to assign values to avoid the SettingWithCopyWarning
for col in ['year', 'times_cited', 'recent_citations', 'num_unique_orgs']:
    X_train.loc[:, col] = pd.to_numeric(X_train[col], errors='coerce')
    X_test.loc[:, col] = pd.to_numeric(X_test[col], errors='coerce')

# Verify final data types
print("Data types in X_train after explicit conversion:\n", X_train.dtypes)
print("Data types in X_test after explicit conversion:\n", X_test.dtypes)


Data types in X_train after explicit conversion:
 year                             int64
times_cited                    float64
recent_citations               float64
is_open_access                   int64
num_unique_orgs                  int64
altmetric_score                float64
field_citation_ratio           float64
citation_count                 float64
relative_citation_ratio        float64
dataset_count                    int64
grant_count                      int64
patent_count                     int64
num_authors                      int64
publication_type_book             bool
publication_type_chapter          bool
publication_type_monograph        bool
publication_type_preprint         bool
publication_type_proceeding       bool
publication_type_seminar          bool
date_diff_days                 float64
num_clinical_trial_ids           int64
num_unique_city_ids              int64
num_unique_country_codes         int64
num_unique_state_codes           int64
dtype: object


In [69]:
# Function to generate a unique identifier for each author
def get_author_id(author):
    if len(author['author_grid_ids']) > 0:  # Check if 'author_grid_ids' is not empty
        return author['author_grid_ids'][0]  # Use the first ID in the list
    else:  # Fallback on 'first_name' and 'last_name'
        return f"{author.get('first_name', '')}_{author.get('last_name', '')}"

In [70]:
authors_ids = authors_info.apply(lambda authors: [get_author_id(author) for author in authors])
print(authors_ids.head())

0             [None_None]
1         [John_McKillop]
2    [Francesca_MCCARTHY]
3             [None_None]
4        [Malcolm_Sawyer]
Name: authors, dtype: object


In [72]:
authors_ids = authors_info.apply(lambda authors: [get_author_id(author) for author in authors])
print(authors_ids.head())

0             [None_None]
1         [John_McKillop]
2    [Francesca_MCCARTHY]
3             [None_None]
4        [Malcolm_Sawyer]
Name: authors, dtype: object


In [73]:
# Combine authors per publication to create a single identifier
authors_groups = authors_ids.apply(lambda ids: "_".join(map(str, ids)))

In [74]:
# Verify there are no NaNs in groups
authors_groups = authors_groups.fillna("Unknown")

In [145]:
# Create deep copies to avoid SettingWithCopyWarning
X_clean = X.copy(deep=True)

In [146]:
# List of problematic columns
problematic_cols = ['year', 'times_cited', 'recent_citations', 'num_unique_orgs']

# Function to display unique types
def display_unique_types(df, cols):
    for col in cols:
        unique_types = df[col].apply(type).unique()
        print(f"\nColumn: '{col}' has the following unique types:")
        for ut in unique_types:
            print(f" - {ut}")

# Inspect unique types in X_clean
print("Inspecting unique types in X_clean:")
display_unique_types(X_clean, problematic_cols)



Inspecting unique types in X_clean:

Column: 'year' has the following unique types:
 - <class 'str'>

Column: 'times_cited' has the following unique types:
 - <class 'str'>
 - <class 'NoneType'>

Column: 'recent_citations' has the following unique types:
 - <class 'str'>
 - <class 'NoneType'>

Column: 'num_unique_orgs' has the following unique types:
 - <class 'str'>


In [135]:
# Display first few entries of problematic columns in X_train_clean
for col in problematic_cols:
    print(f"\nFirst 10 entries in '{col}' of X_train_clean:")
    print(X_train_clean[col].head(10))



First 10 entries in 'year' of X_train_clean:
1     2023.0
2     2024.0
4     2024.0
5     2023.0
8     2024.0
9     2023.0
12    2023.0
14    2024.0
17    2023.0
18    2020.0
Name: year, dtype: object

First 10 entries in 'times_cited' of X_train_clean:
1     0.0
2     0.0
4     0.0
5     0.0
8     0.0
9     0.0
12    0.0
14    0.0
17    0.0
18    0.0
Name: times_cited, dtype: object

First 10 entries in 'recent_citations' of X_train_clean:
1     0.0
2     0.0
4     0.0
5     0.0
8     0.0
9     0.0
12    0.0
14    0.0
17    0.0
18    0.0
Name: recent_citations, dtype: object

First 10 entries in 'num_unique_orgs' of X_train_clean:
1     0.0
2     0.0
4     0.0
5     0.0
8     0.0
9     0.0
12    0.0
14    0.0
17    0.0
18    0.0
Name: num_unique_orgs, dtype: object


In [147]:
# Convert problematic columns to numeric types using .astype()
for col in problematic_cols:
    try:
        X_clean[col] = X_clean[col].astype(float)
        print(f"Successfully converted '{col}' to float.")
    except ValueError as e:
        print(f"Error converting '{col}': {e}")


Successfully converted 'year' to float.
Successfully converted 'times_cited' to float.
Successfully converted 'recent_citations' to float.
Successfully converted 'num_unique_orgs' to float.


In [180]:
from xgboost import XGBRegressor

# Define the XGBoost model
model = XGBRegressor(missing=np.nan, n_jobs=-1)

# Fit the model
try:
    model.fit(X_clean, y)
    print("\nXGBoost model fitted successfully without Bayesian optimization.")
except Exception as e:
    print("\nError during XGBoost model fitting:", e)


XGBoost model fitted successfully without Bayesian optimization.


In [184]:
model = XGBRegressor(missing=np.nan, n_jobs=-1, enable_categorical=True)
# Define parameter search space for Bayesian optimization
from skopt.space import Integer, Real
param_space = {
    'n_estimators': Integer(50, 5000),
    'max_depth': Integer(3, 15),
    'learning_rate': Real(0.01, 0.2, 'log-uniform'),
    'subsample': Real(0.5, 1.0),
}

In [182]:
outer_cv = TimeSeriesSplit(n_splits=4)
inner_cv = TimeSeriesSplit(n_splits=4)

In [195]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
import pickle

# Set to True to resume from checkpoint, False to start fresh
resume_from_checkpoint = True

# Filepaths for checkpoint files
results_file = "/work/teghipco/LLMs/Biobert/bayesian_optimization_results_checkpoint_altmetric.csv"
predictions_file = "/work/teghipco/LLMs/Biobert/bayesian_optimization_predictions_checkpoint_altmetric.csv"
state_file = "/work/teghipco/LLMs/Biobert/bayesian_optimization_state_altmetric.pkl"

# Initialize lists to store RMSE, MAE, and predictions for each outer fold
outer_rmse_scores = []
outer_mae_scores = []
all_predictions = []

# Resume or start fresh based on the switch
if resume_from_checkpoint and os.path.exists(state_file):
    # Load previous state if checkpoint exists and resuming
    with open(state_file, "rb") as f:
        checkpoint_data = pickle.load(f)
        start_fold = checkpoint_data['start_fold']
        outer_rmse_scores = checkpoint_data['outer_rmse_scores']
        outer_mae_scores = checkpoint_data['outer_mae_scores']
        results_df = pd.read_csv(results_file)
        all_predictions_df = pd.read_csv(predictions_file)
        all_predictions = [all_predictions_df]  # initialize with existing predictions
    print(f"Resuming from fold {start_fold + 1}")
    print(f"Checkpoint loaded - Start fold: {start_fold}, RMSE scores: {outer_rmse_scores}, MAE scores: {outer_mae_scores}")
else:
    # Start fresh, initialize from scratch
    start_fold = 0
    results_df = pd.DataFrame(columns=["Outer Fold", "RMSE", "MAE"])

Resuming from fold 5
Checkpoint loaded - Start fold: 4, RMSE scores: [np.float64(109.86921728414308), np.float64(63.47812610292604), np.float64(68.14318936758099), np.float64(49.87415927697745)], MAE scores: [np.float64(3.188687765043222), np.float64(2.155909136049051), np.float64(1.3475056230235132), np.float64(1.8557800355756917)]


In [196]:
# Outer loop: Train on consecutive years, test on future years
for outer_fold, (train_index, test_index) in enumerate(outer_cv.split(X_clean)):
    if outer_fold < start_fold:
        continue  # Skip completed folds

    # Identify unique authors in the test set
    test_authors = set(authors_groups.iloc[test_index])

    # Filter training set to exclude any authors that appear in the test set
    filtered_train_index = [i for i in train_index if authors_groups.iloc[i] not in test_authors]
    
    # Prepare training and test sets
    X_train_clean, X_test_clean = X_clean.iloc[filtered_train_index], X_clean.iloc[test_index]
    y_train, y_test = y.iloc[filtered_train_index], y.iloc[test_index]
    filtered_train_authors = authors_groups.iloc[filtered_train_index]
    
    # Scale X and y
    X_scaler = StandardScaler()
    y_scaler = StandardScaler()

    X_train_scaled = X_scaler.fit_transform(X_train_clean)
    X_test_scaled = X_scaler.transform(X_test_clean)
    y_train_scaled = y_scaler.fit_transform(y_train.values.reshape(-1, 1)).flatten()
    y_test_scaled = y_scaler.transform(y_test.values.reshape(-1, 1)).flatten()
    
    # Set up inner GroupKFold cross-validation using the filtered author groups
    inner_cv = GroupKFold(n_splits=3)

    # Define Bayesian optimization
    opt = BayesSearchCV(
        model,
        param_space,
        cv=inner_cv.split(X_train_scaled, y_train_scaled, groups=filtered_train_authors),
        n_iter=80,
        scoring='neg_mean_squared_error',
        n_jobs=-1,
        verbose=3,
    )

    # Fit the Bayesian optimization
    print(f"Starting Bayesian optimization for outer fold {outer_fold + 1}")
    opt.fit(X_train_scaled, y_train_scaled)

    # Evaluate on the outer test set
    y_pred_scaled = opt.predict(X_test_scaled)
    y_pred = y_scaler.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()
    y_test_orig = y_scaler.inverse_transform(y_test_scaled.reshape(-1, 1)).flatten()

    # Calculate RMSE and MAE
    rmse = np.sqrt(np.mean((y_pred - y_test_orig) ** 2))
    mae = mean_absolute_error(y_test_orig, y_pred)
    
    # Save results for this fold
    outer_rmse_scores.append(rmse)
    outer_mae_scores.append(mae)
    results_df = pd.concat([results_df, pd.DataFrame([{"Outer Fold": outer_fold + 1, "RMSE": rmse, "MAE": mae}])], ignore_index=True)

    # Save predictions
    fold_predictions = pd.DataFrame({
        "Outer Fold": outer_fold + 1,
        "Publication ID": pub_id.iloc[test_index],
        "Actual Altmetric Score": y_test_orig,
        "Predicted Altmetric Score": y_pred
    })
    all_predictions.append(fold_predictions)

    # Save checkpoints after each fold
    results_df.to_csv(results_file, index=False)
    all_predictions_df = pd.concat(all_predictions, ignore_index=True)
    all_predictions_df.to_csv(predictions_file, index=False)

    # Update the checkpoint data
    checkpoint_data = {
        'start_fold': outer_fold + 1,
        'outer_rmse_scores': outer_rmse_scores,
        'outer_mae_scores': outer_mae_scores,
    }
    with open(state_file, "wb") as f:
        pickle.dump(checkpoint_data, f)

    print(f"Outer fold {outer_fold + 1} completed - RMSE: {rmse}, MAE: {mae}")

# After all folds are done, clean up checkpoint files if starting fresh
if not resume_from_checkpoint:
    for file in [state_file, results_file, predictions_file]:
        if os.path.exists(file):
            os.remove(file)

print("All folds completed. Checkpoint files cleaned up if starting fresh.")

All folds completed. Checkpoint files cleaned up if starting fresh.


In [197]:
from scipy.stats import pearsonr

# Concatenate all actual and predicted values across all folds
all_actual_values = pd.concat([fold["Actual Altmetric Score"] for fold in all_predictions], ignore_index=True)
all_predicted_values = pd.concat([fold["Predicted Altmetric Score"] for fold in all_predictions], ignore_index=True)

# Calculate the overall Pearson correlation coefficient
correlation, _ = pearsonr(all_actual_values, all_predicted_values)

# Print the correlation
print(f"\nOverall Pearson correlation between actual and predicted Altmetric scores: {correlation}")


Overall Pearson correlation between actual and predicted Altmetric scores: 0.901786741971732


In [198]:
print(all_actual_values.shape)

(2383420,)


In [199]:
print(y.shape)

(2979275,)
